In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torch import nn, optim
import numpy as np


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize with mean=0.5 and std=0.5
])

# Download and load the training data
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Download and load the test data
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Example: Display the size of the training set
print(f"Number of training samples: {len(train_dataset)}")


Number of training samples: 60000


In [3]:

print("Original labels (before noise):", train_dataset.targets[:10].tolist())

# Function to add label noise by shuffling labels
def add_label_noise(dataset, noise_fraction=0.1):
    """
    Adds label noise by shuffling labels for a fraction of the dataset.

    Args:
        dataset: A PyTorch dataset object (e.g., MNIST).
        noise_fraction: Fraction of data to corrupt (0 <= noise_fraction <= 1).

    Returns:
        The dataset with shuffled labels for the specified fraction.
    """
    # Total number of samples
    num_samples = len(dataset)
    
    # Number of labels to corrupt
    num_noisy_labels = int(noise_fraction * num_samples)
    
    # Get the indices to shuffle
    indices = np.random.choice(num_samples, num_noisy_labels, replace=False)
    
    # Shuffle the labels
    noisy_targets = dataset.targets.clone()  # Clone the targets to avoid modifying the original directly
    noisy_targets[indices] = torch.randint(0, 10, size=(num_noisy_labels,))  # Random labels in the range [0, 9]
    
    # Update the dataset's targets
    dataset.targets = noisy_targets
    return dataset



Original labels (before noise): [5, 0, 4, 1, 9, 2, 1, 3, 1, 4]


In [7]:
subset_size = 10000  # Use only 10,000 samples
indices = np.random.choice(len(train_dataset), subset_size, replace=False)  # Randomly select indices
train_subset = Subset(train_dataset, indices)
# Add noise to the training dataset (e.g., 20% label noise)
train_subset = add_label_noise(train_subset, noise_fraction=0.2)


# Create data loaders
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


AttributeError: 'Subset' object has no attribute 'targets'

In [8]:
class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),  # First layer with 512 neurons
            nn.ReLU(),
            #nn.Dropout(0.3),       # Dropout for regularization
            nn.Linear(512, 256),   # Second layer with 256 neurons
            nn.ReLU(),
            #nn.Dropout(0.3),
            nn.Linear(256, 128),   # Third layer with 128 neurons
            nn.ReLU(),
            #nn.Dropout(0.3),
            nn.Linear(128, 64),    # Fourth layer with 64 neurons
            nn.ReLU(),
            nn.Linear(64, 10)      # Output layer for 10 classes
        )
    
    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        return self.model(x)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeepNN().to(device)
criterion = nn.CrossEntropyLoss()

# Define optimizers
optimizers = {
    'SGD': optim.SGD(model.parameters(), lr=0.01, momentum=0.9),
    'Adam': optim.Adam(model.parameters(), lr=0.001),
    'RMSprop': optim.RMSprop(model.parameters(), lr=0.001)
}

# Training function
def train_model(optimizer_name, optimizer, num_epochs=30):
    print(f"\nTraining with {optimizer_name}...")
    model.apply(lambda m: m.reset_parameters() if hasattr(m, 'reset_parameters') else None)
    optimizer = optimizer
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Accumulate metrics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        # Calculate training metrics
        train_accuracy = 100 * correct_train / total_train
        train_loss = running_loss / len(train_loader)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Training Loss: {train_loss:.4f}, "
              f"Training Accuracy: {train_accuracy:.2f}%")
    
    # Evaluate the model on the test set
    evaluate_model()

# Evaluation function
def evaluate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Train the model with each optimizer
for opt_name, opt in optimizers.items():
    train_model(opt_name, opt)


Training with SGD...
Epoch [1/30], Training Loss: 1.7370, Training Accuracy: 40.04%
Epoch [2/30], Training Loss: 0.5442, Training Accuracy: 82.34%
Epoch [3/30], Training Loss: 0.3641, Training Accuracy: 88.67%
Epoch [4/30], Training Loss: 0.2860, Training Accuracy: 91.36%
Epoch [5/30], Training Loss: 0.2255, Training Accuracy: 93.06%
Epoch [6/30], Training Loss: 0.1819, Training Accuracy: 94.31%
Epoch [7/30], Training Loss: 0.1538, Training Accuracy: 95.16%
Epoch [8/30], Training Loss: 0.1468, Training Accuracy: 95.36%
Epoch [9/30], Training Loss: 0.1221, Training Accuracy: 96.18%
Epoch [10/30], Training Loss: 0.1045, Training Accuracy: 96.72%
Epoch [11/30], Training Loss: 0.0956, Training Accuracy: 97.02%
Epoch [12/30], Training Loss: 0.0893, Training Accuracy: 97.12%
Epoch [13/30], Training Loss: 0.0738, Training Accuracy: 97.69%
Epoch [14/30], Training Loss: 0.0625, Training Accuracy: 97.94%
Epoch [15/30], Training Loss: 0.0488, Training Accuracy: 98.41%
Epoch [16/30], Training Los

Original labels (before noise): [5, 2, 0, 9, 4, 6, 8, 6, 1, 6]
Labels after adding noise: [5, 9, 0, 9, 4, 9, 5, 6, 1, 6]
